In [ ]:
from flask import Flask, app, request, jsonify
import time
from core import get_prediction

In [ ]:
KERNEL_SIZE = 224
HEALTHY_THRESHOLD=50/100
ARCHITECTURE = 'custom'
RESCALING_FACTOR=1/255
short_symptom_labels = {
    0: 'FMD',
    1: 'LSD',
    2: 'NOR'
}
model = tf.keras.models.load_model(f"./{ARCHITECTURE}_model")

'''
    Resize both dimensions to multiples of the kernel size
    to allow splitting the image into tiles
'''


def resize_image(img):
    return img.resize((KERNEL_SIZE, KERNEL_SIZE), Image.ANTIALIAS)

'''
    Splitting the image into tiles of the kernel size
    then reshaping it
'''
def split_and_reshape(img):
    height, width, channels = img.shape
    img = img.reshape(1, KERNEL_SIZE,
                      1, KERNEL_SIZE, channels)
    img = img.swapaxes(1, 2)
    img = img.reshape(-1, img.shape[-3], img.shape[-2], img.shape[-1])
    return img
'''
    The most frequent element is the most likely
    NOTE: "Healthy" tile have to be handled carefully.
    No matter how high the count of "healthy" tiles are, if there's any spot that's
    not healthy, the whole leaf might not be healthy.
    The threshold is set to 92% (>=92% Healthy is considered actually healthy).
'''


def get_prediction(file):
    img = Image.open(file)
    img = resize_image(img)
    original_img = np.asarray(img)
    rescaled_img = original_img * RESCALING_FACTOR
    rescaled_img = split_and_reshape(rescaled_img)
    model_prediction = model.predict(rescaled_img)
    classification = np.argmax(model_prediction, axis=1)
    classification_size=len(classification)
    counter = Counter(classification)
    confidence = 0
    for i in range(classification_size):
        confidence += model_prediction[i][classification[i]]
    confidence /= classification_size
    return short_symptom_labels[counter.most_common(1)[0][0]], str(round(confidence * 100, 2))

In [39]:
path = "input/cattle-disease/test/NOR/2-4.jpg"

In [40]:
get_prediction(path)

C:\Users\Tho Le\AppData\Local\Temp\ipykernel_52572\124436196.py:19: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  return img.resize((KERNEL_SIZE, KERNEL_SIZE), Image.ANTIALIAS)


('FMD', '99.99')

In [41]:
prediction, confidence = get_prediction(path)

C:\Users\Tho Le\AppData\Local\Temp\ipykernel_52572\124436196.py:19: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  return img.resize((KERNEL_SIZE, KERNEL_SIZE), Image.ANTIALIAS)


In [42]:
print(prediction)

FMD


In [ ]:
classification_size = len(classification)
    counter = Counter(classification)
    confidence = 0
    for i in range(classification_size):
        confidence += model_prediction[i][classification[i]]
    confidence /= classification_size

    if counter[0] >= classification_size * HEALTHY_THRESHOLD:
        return short_symptom_labels[0], str(round(confidence * 100, 2))
    else:
        del counter[0]

    return short_symptom_labels[counter.most_common(1)[0][0]], str(round(confidence * 100, 2))